In [3]:
#from pysat.examples.fm import FM
#from pysat.formula import WCNF
#import numpy as np
import collections
# Import Python wrapper for or-tools CP-SAT solver.
from ortools.sat.python import cp_model

def OpenShopSat():
    # Create the model
    model = cp_model.CpModel()

    ##################### DATA #####################
    
    # Task = (processor_id_assigned, unit_time)
    #jobs_values = [[(0, 1), (1, 1), (2, 1), (3,1)],  # Job0
     #             [(0, 1), (2, 1), (1, 1)],          # Job1
      #            [(3, 1), (1, 1)],                  # Job2
       #           [(2,1), (1,1), (2,1)]]             # Job3
    
    # Other examples
    #jobs_values = [[(0, 1), (1, 1), (2, 1)], [(4, 1), (3, 1)], [(1,1), (0, 1), (3,1)], [(1,1), (2,1), (4,1)], [(3,1)]]
    jobs_values = [[(0, 1)], [(2, 1), (2, 1), (3, 1), (3, 1)], [(1,1), (0, 1), (1,1)], [(0,1), (2,1), (3,1), (3,1), (1,1), (0,1)]]
    # jobs_values = [[(4, 1), (3, 1), (2, 1), (1,1)], [(4, 1), (0,1), (0, 1), (0,1), (4,1)], [(1,1)], [(1, 1), (1,1), (2,1), (0,1), (3,1)], 
    #                [(2,1), (3,1)], [(1,1), (2,1), (1,1), (4,1), (2,1), (0,1), (0,1)]]
    
    # Compute sum of all unit_times
    k = sum(task[1] for job in jobs_values for task in job)  
    #print(k)
    
    # Compute number of processors 
    number_processors = max(task[0] for job in jobs_values for task in job) + 1
    processors = range(number_processors)
    #print(number_processors)

    ##################### CREATE VARIABLES #####################
    
    # Create Named_tuple to store info 
    task_info = collections.namedtuple('task_info', 'start end interval')
    #print(task_info)
    # Create Named_tuple to handle info
    assigned_task_info = collections.namedtuple('assigned_task_info', 'start job index duration')
    #print(assigned_task_info)
    
    all_the_tasks = {}
    processors_intervals = collections.defaultdict(list)

    # Create job intervals and put in processor lists
    for job_id, job in enumerate(jobs_values):
        for task_id, task in enumerate(job):
            processor_id = task[0]
            duration = task[1]      # always 1
            s = '_%d_%d' % (job_id, task_id)
            start_var = model.NewIntVar(0, k, 'start' + s) 
            end_var = model.NewIntVar(0, k, 'end' + s)  
            interval_var = model.NewIntervalVar(start_var, duration, end_var,'interval' + s)
            all_the_tasks[job_id, task_id] = task_info(start=start_var, end=end_var, interval=interval_var)
            #print(all_the_tasks) #----> a ogni task creo un inizio, una fine, e un intervallo di una unità
            processors_intervals[processor_id].append(interval_var)
            #print(processors_intervals) #-----> associa a ogni processor degli intervalli (senza valori) delle task a lui affidate
    
    ##################### CONSTRAINTS #####################
    
    # No overlapping
    for processor_id in processors:
        model.AddNoOverlap(processors_intervals[processor_id])

    # Time priority of parts of a job
    for job_id, job in enumerate(jobs_values):
        for task_id in range(len(job) - 1):
            model.Add(all_the_tasks[job_id, task_id + 1].start >= all_the_tasks[job_id, task_id].end)

    # Time as "objective function"
    time = model.NewIntVar(0, k, 'time')
    model.AddMaxEquality(time, [all_the_tasks[job_id, len(job) - 1].end for job_id, job in enumerate(jobs_values)]) 
    model.Minimize(time) 

    
    # Solve model
    solver = cp_model.CpSolver()
    solution = solver.Solve(model)

    ##################### OUTPUT #####################
    
    # Create one list of assigned tasks for each processor
    if solution == cp_model.OPTIMAL:
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_values):
            for task_id, task in enumerate(job):
                processor_id = task[0]
                assigned_jobs[processor_id].append(assigned_task_info(start=solver.Value(
                    all_the_tasks[job_id, task_id].start), job=job_id, index=task_id, duration=task[1]))
        #print(assigned_jobs)   #----->associa a ogni processore le sue task, con i valori degli intervalli perchè ho calcolato con solver
                
        # Create output 
        output = ''
        for processor_id in processors:
            assigned_jobs[processor_id].sort()
            main_line = 'Processor ' + str(processor_id) + ': '
            line_time = '              '

            for assigned_task in assigned_jobs[processor_id]:
                a = 'job_%d_%d' % (assigned_task.job, assigned_task.index)
                main_line += '%-11s' %a
                start = assigned_task.start
                duration = assigned_task.duration
                b = '[%d,%d]' % (start, start + duration)
                line_time += '%-11s' %b 
                
            line_time += '\n'
            main_line += '\n'
            output += main_line
            output += line_time

        # Print solution 
        print(output)
        print('Number of processors: ', number_processors)
        print('Optimal Schedule Time: %i' % solver.ObjectiveValue())
        
OpenShopSat()

Processor 0: job_3_0    job_0_0    job_2_1    job_3_5    
              [0,1]      [1,2]      [2,3]      [5,6]      
Processor 1: job_2_0    job_2_2    job_3_4    
              [0,1]      [3,4]      [4,5]      
Processor 2: job_1_0    job_3_1    job_1_1    
              [0,1]      [1,2]      [2,3]      
Processor 3: job_3_2    job_3_3    job_1_2    job_1_3    
              [2,3]      [3,4]      [4,5]      [5,6]      

Number of processors:  4
Optimal Schedule Time: 6
